# importing data

In [1]:
import scipy.io as sio
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%pprint

Pretty printing has been turned OFF


In [3]:
start_dir = "../../"+os.path.relpath(os.path.expanduser("~"))+"/Volumes/project/Hanson"

In [4]:
floc = "/External_MRI_Projects/NKI_Rockland/Neuroimaging/BIDS/derivatives/DWI_mrtrix_preprocessed"

In [5]:
[ x for x in os.listdir(start_dir+floc) if x.endswith("mat") ]

['Fingerprint_test_connectometry.qa.db.fib.gz.vec0.mat', 'FingerPrintTest.N677.connectometry.qa.db.fib.gz.vec0.mat', 'FingerPrintTest.N677.connectometry.qa.db.fib.gz.vec1.mat']

In [6]:
test_fn = '/Fingerprint_test_connectometry.qa.db.fib.gz.vec0.mat'
real_fn1 = '/FingerPrintTest.N677.connectometry.qa.db.fib.gz.vec0.mat'
real_fn2 = '/FingerPrintTest.N677.connectometry.qa.db.fib.gz.vec1.mat'

In [7]:
# matfile1 = sio.loadmat(start_dir+floc+real_fn1)
# matfile2 = sio.loadmat(start_dir+floc+real_fn2)

In [8]:
# joblib is the same as pickle but more efficient for stuff w numpy arrays
from joblib import dump, load

In [9]:
# dump(matfile1, '../files/mat1.joblib',compress=3) 
# dump(matfile2, '../files/mat2.joblib',compress=3) 

In [10]:
matfile1 = load('../files/mat1.joblib') 
matfile2 = load('../files/mat2.joblib') 

In [11]:
list(matfile1.keys())[:20]

['subject_names', 'subject0', 'subject1', 'subject2', 'subject3', 'subject4', 'subject5', 'subject6', 'subject7', 'subject8', 'subject9', 'subject10', 'subject11', 'subject12', 'subject13', 'subject14', 'subject15', 'subject16', 'subject17', 'subject18']

In [12]:
list(matfile1.keys())[-20:]

['subject384', 'subject385', 'subject386', 'subject387', 'subject388', 'subject389', 'subject390', 'subject391', 'subject392', 'subject393', 'subject394', 'subject395', 'subject396', 'subject397', 'subject398', 'subject399', 'dimension', 'voxel_location', 'mni_location', 'fiber_direction']

In [16]:
os.listdir("../files")

['fingerprint_31_gs.joblib', 'fingerprint_model_gs.joblib', 'fingerprint_32_gs_r2.joblib', 'fingerprinty_30.joblib', 'fingerprinty_32.joblib', 'mat2.joblib', 'fingerprint_cv_scores.joblib', 'fingerprintY.joblib', 'fingerprint_30_gs.joblib', 'fingerprint_all_gs.joblib', 'fingerprint_29_gs.joblib', 'fingerprint_svr_gs.joblib', 'mat1.joblib', 'fingerprint_model.joblib', 'NKI_IRI_DWI_combined.csv', 'fingerprinty_31.joblib', 'fingerprint_29_gs_r2.joblib', 'NKIRS_Impulsivity.csv', 'fingerprintX_pca.joblib', 'fingerprint_32_gs.joblib', 'fingerprinty_29.joblib', 'fingerprintX.joblib', 'fingerprint_31_gs_r2.joblib', 'NKI_fingerprint_test.csv', 'fingerprint_30_gs_r2.joblib']

In [36]:
# phenotypic data
pheno_df = pd.read_csv("../files/NKI_IRI_DWI_combined.csv")
impulse_df = pd.read_csv("../files/NKIRS_Impulsivity.csv")

In [37]:
pheno_df["sub"]=pheno_df.Identifiers_alt.map(lambda x: x.split("_")[0].split("-")[-1])
pheno_df["ses"]=pheno_df.Identifiers_alt.map(lambda x: x.split("_")[1].split("-")[-1])

In [38]:
pheno_df.head()

,Identifiers_alt,age,sex,ipri_29,ipri_30,ipri_31,ipri_32,sub,ses
0,sub-A00023510_ses-BAS1,23,0,17,21,19,17,A00023510,BAS1
1,sub-A00027439_ses-BAS1,15,0,17,11,14,8,A00027439,BAS1
2,sub-A00027443_ses-BAS1,15,0,11,9,16,5,A00027443,BAS1
3,sub-A00027544_ses-BAS1,22,0,22,19,24,16,A00027544,BAS1
4,sub-A00028150_ses-BAS1,44,1,12,18,19,17,A00028150,BAS1


In [83]:
len(impulse_df)/len(pheno_df)

1.7799113737075332

In [39]:
impulse_df["sub"]=impulse_df.Identifiers.map(lambda x: x.split(",")[0])
impulse_df["ses"]=impulse_df.Identifiers.map(lambda x: x.split(",")[1])

In [40]:
impulse_df.head()

,Identifiers,upps_65,sub,ses
0,"A00008326,BAS1",109,A00008326,BAS1
1,"A00008326,BAS2",97,A00008326,BAS2
2,"A00008399,BAS1",134,A00008399,BAS1
3,"A00010893,BAS1",122,A00010893,BAS1
4,"A00013809,BAS1",107,A00013809,BAS1


In [85]:
pheno_df.loc[10,"sub"]

'A00028352'

In [84]:
impulse_df["sub"]

0       A00008326
1       A00008326
2       A00008399
3       A00010893
4       A00013809
          ...    
1200    A00088403
1201    A00088408
1202    A00088716
1203    A00089407
1204    A00089408
Name: sub, Length: 1205, dtype: object

In [79]:
valid_subs = []
for i in pheno_df.index:
    valid_subs.append(pheno_df.loc[i,"sub"] in impulse_df["sub"])

In [80]:
valid_subs

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False

In [78]:
'A000023510' in impulse_df["sub"]

False

In [72]:
pheno_df.loc[0,"ses"]

'BAS1'

In [65]:
def getImpulse(subval, sesval):
    return impulse_df.loc[impulse_df['sub'].eq(subval) & impulse_df['ses'].eq(sesval)].upps_65[0]

In [73]:
getImpulse("A00023510", "BAS1")

KeyError: 0

In [27]:
impulse_df.head()

,Identifiers,upps_65,sub,ses
0,"A00008326,BAS1",109,A00008326,BAS1
1,"A00008326,BAS2",97,A00008326,BAS2
2,"A00008399,BAS1",134,A00008399,BAS1
3,"A00010893,BAS1",122,A00010893,BAS1
4,"A00013809,BAS1",107,A00013809,BAS1


In [15]:
pheno_df.describe()

,age,sex,ipri_29,ipri_30,ipri_31,ipri_32
count,677.000000,677.000000,677.000000,677.000000,677.000000,677.000000
mean,32.324963,0.604136,15.877400,14.422452,20.404727,11.707533
std,13.110167,0.489397,4.216554,6.009014,5.072420,5.204699
min,13.000000,0.000000,3.000000,0.000000,0.000000,1.000000
25%,21.000000,0.000000,13.000000,10.000000,17.000000,8.000000
50%,30.000000,1.000000,16.000000,15.000000,21.000000,12.000000
75%,45.000000,1.000000,19.000000,19.000000,24.000000,15.000000
max,54.000000,1.000000,24.000000,28.000000,28.000000,31.000000


# modeling
## data prep
### X - fingerprint data

In [16]:
# just the subjects
subj1 = [x for x in list(matfile1.keys()) if "sub" in x and "_" not in x]
subj2 = [x for x in list(matfile2.keys()) if "sub" in x and "_" not in x]

In [17]:
len(subj1)

400

In [18]:
len(subj2)

277

In [19]:
X = []
for sub in subj1:
    X.append(matfile1[sub][0])
for sub in subj2:
    X.append(matfile2[sub][0])

In [20]:
X = np.array(X)
X

array([[0.13154687, 0.14433745, 0.1312975 , ..., 0.03943831, 0.02088308,
        0.05409636],
       [0.8178366 , 0.74562216, 0.72506034, ..., 0.34529442, 0.41764247,
        0.56341   ],
       [0.50131667, 0.47282678, 0.5085435 , ..., 0.1921919 , 0.2769494 ,
        0.2239489 ],
       ...,
       [2.193593  , 1.8504024 , 2.1619565 , ..., 0.5081969 , 0.4759148 ,
        0.6066462 ],
       [0.2378414 , 0.17248623, 0.23666348, ..., 0.8264794 , 0.7652172 ,
        0.7216453 ],
       [0.32788658, 0.35372517, 0.36588156, ..., 0.14133348, 0.10620189,
        0.06241945]], dtype=float32)

In [21]:
# already saved
# dump(X, '../files/fingerprintX.joblib',compress=3) 
# dump(y, '../files/fingerprintY.joblib',compress=3) 

In [22]:
len(X[0])

319861

In [23]:
# N_COMPONENTS = len(X)

In [24]:
# PCA is preferable but depending on the shape of the data i might use truncatedSVD
# from sklearn.decomposition import PCA, TruncatedSVD

In [25]:
# pca = PCA(n_components=N_COMPONENTS, random_state=0)
# X_pca = pca.fit_transform(X)

In [26]:
# sweet
# len(X_pca[0])

In [27]:
# dump(X_pca, '../files/fingerprintX_pca.joblib',compress=3) 

In [28]:
X_pca = load('../files/fingerprintX_pca.joblib') 

### y(s) - pheno data

In [29]:
pheno_df.head()

,Identifiers_alt,age,sex,ipri_29,ipri_30,ipri_31,ipri_32
0,sub-A00023510_ses-BAS1,23,0,17,21,19,17
1,sub-A00027439_ses-BAS1,15,0,17,11,14,8
2,sub-A00027443_ses-BAS1,15,0,11,9,16,5
3,sub-A00027544_ses-BAS1,22,0,22,19,24,16
4,sub-A00028150_ses-BAS1,44,1,12,18,19,17


In [32]:
y_29 = np.array(pheno_df.ipri_29)
y_30 = np.array(pheno_df.ipri_30)
y_31 = np.array(pheno_df.ipri_31)
y_32 = np.array(pheno_df.ipri_32)

In [33]:
dump(y_29, '../files/fingerprinty_29.joblib',compress=3) 
dump(y_30, '../files/fingerprinty_30.joblib',compress=3) 
dump(y_31, '../files/fingerprinty_31.joblib',compress=3) 
dump(y_32, '../files/fingerprinty_32.joblib',compress=3) 

['../files/fingerprinty_32.joblib']

## fitting prelim models

In [63]:
len(X_pca)

677

In [65]:
from sklearn.model_selection import train_test_split

In [66]:
X_train29, X_test29, y_train29, y_test29 = train_test_split(
  X_pca, y_29, test_size = 1/3, random_state = 0)
  
X_train30, X_test30, y_train30, y_test30 = train_test_split(
  X_pca, y_30, test_size = 1/3, random_state = 0)
  
X_train31, X_test31, y_train31, y_test31 = train_test_split(
  X_pca, y_31, test_size = 1/3, random_state = 0)
  
X_train32, X_test32, y_train32, y_test32 = train_test_split(
  X_pca, y_32, test_size = 1/3, random_state = 0)

In [77]:
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
scorer = make_scorer(mean_squared_error, greater_is_better=False)

### IPRI 29

In [90]:
fingerprint_29_gs = load('../files/fingerprint_29_gs_r2.joblib')

In [79]:
#svr_gs.cv_results_
fingerprint_29_gs.best_params_

{'model': GradientBoostingRegressor(learning_rate=0.001, max_depth=2, n_estimators=1000,
                          n_iter_no_change=50, random_state=0), 'model__learning_rate': 0.001, 'model__max_depth': 2, 'model__n_estimators': 1000, 'model__n_iter_no_change': 50}

In [80]:
means29 = fingerprint_29_gs.cv_results_["mean_test_score"]
stds29 = fingerprint_29_gs.cv_results_["std_test_score"]
results29 = sorted(list(zip(means29, stds29, fingerprint_29_gs.cv_results_["params"])), 
                 key = lambda x: x[0], reverse=True)

In [81]:
for mean, std, params in results29[:10]:
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()

0.003 (+/-0.023) for {'model': GradientBoostingRegressor(learning_rate=0.001, max_depth=2, n_estimators=1000,
                          n_iter_no_change=50, random_state=0), 'model__learning_rate': 0.001, 'model__max_depth': 2, 'model__n_estimators': 1000, 'model__n_iter_no_change': 50}

0.002 (+/-0.031) for {'model': ElasticNet(random_state=0), 'model__alpha': 100.0, 'model__l1_ratio': 0.1, 'model__selection': 'cyclic'}

0.002 (+/-0.031) for {'model': ElasticNet(random_state=0), 'model__alpha': 100.0, 'model__l1_ratio': 0.1, 'model__selection': 'random'}

0.001 (+/-0.011) for {'model': GradientBoostingRegressor(learning_rate=0.001, max_depth=2, n_estimators=1000,
                          n_iter_no_change=50, random_state=0), 'model__learning_rate': 0.01, 'model__max_depth': 2, 'model__n_estimators': 10, 'model__n_iter_no_change': 50}

0.000 (+/-0.010) for {'model': GradientBoostingRegressor(learning_rate=0.001, max_depth=2, n_estimators=1000,
                          n_iter_no_chang

In [84]:
labels29 = fingerprint_29_gs.predict(X_test29)

In [85]:
r2_score(y_test29, labels29)

0.0014816375233402956

### IPRI 30

In [91]:
fingerprint_30_gs_r2 = load('../files/fingerprint_30_gs_r2.joblib')
fingerprint_30_gs_mse = load('../files/fingerprint_30_gs.joblib')

In [105]:
print(fingerprint_30_gs_r2.best_params_)
print(fingerprint_30_gs_mse.best_params_)

{'model': GradientBoostingRegressor(learning_rate=0.01, max_depth=2, n_iter_no_change=50,
                          random_state=0), 'model__learning_rate': 0.01, 'model__max_depth': 2, 'model__n_estimators': 100, 'model__n_iter_no_change': 50}
{'model': GradientBoostingRegressor(learning_rate=0.01, max_depth=2, n_iter_no_change=50,
                          random_state=0), 'model__learning_rate': 0.01, 'model__max_depth': 2, 'model__n_estimators': 100, 'model__n_iter_no_change': 50}


In [106]:
means30 = fingerprint_30_gs_r2.cv_results_["mean_test_score"]
stds30 = fingerprint_30_gs_r2.cv_results_["std_test_score"]
results30 = sorted(list(zip(means30, stds30, fingerprint_30_gs_r2.cv_results_["params"])), 
                 key = lambda x: x[0],reverse=True)

In [107]:
for mean, std, params in results30[:10]:
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()

0.003 (+/-0.020) for {'model': GradientBoostingRegressor(learning_rate=0.01, max_depth=2, n_iter_no_change=50,
                          random_state=0), 'model__learning_rate': 0.01, 'model__max_depth': 2, 'model__n_estimators': 100, 'model__n_iter_no_change': 50}

0.003 (+/-0.020) for {'model': GradientBoostingRegressor(learning_rate=0.01, max_depth=2, n_iter_no_change=50,
                          random_state=0), 'model__learning_rate': 0.01, 'model__max_depth': 2, 'model__n_estimators': 1000, 'model__n_iter_no_change': 50}

0.001 (+/-0.009) for {'model': GradientBoostingRegressor(learning_rate=0.01, max_depth=2, n_iter_no_change=50,
                          random_state=0), 'model__learning_rate': 0.001, 'model__max_depth': 2, 'model__n_estimators': 1000, 'model__n_iter_no_change': 50}

-0.000 (+/-0.004) for {'model': GradientBoostingRegressor(learning_rate=0.01, max_depth=2, n_iter_no_change=50,
                          random_state=0), 'model__learning_rate': 0.01, 'model__max

In [103]:
labels30 = fingerprint_30_gs_r2.predict(X_test30)

In [104]:
r2_score(y_test30, labels30)

-0.01939400737556385

### IPRI 31

In [100]:
fingerprint_31_gs_r2 = load('../files/fingerprint_31_gs_r2.joblib')
fingerprint_31_gs_mse = load('../files/fingerprint_31_gs.joblib')

In [101]:
print(fingerprint_31_gs_r2.best_params_)
print(fingerprint_31_gs_mse.best_params_)

{'model': ElasticNet(alpha=100.0, l1_ratio=0.1, random_state=0), 'model__alpha': 100.0, 'model__l1_ratio': 0.1, 'model__selection': 'cyclic'}
{'model': ElasticNet(alpha=100.0, l1_ratio=0.1, random_state=0), 'model__alpha': 100.0, 'model__l1_ratio': 0.1, 'model__selection': 'cyclic'}


In [109]:
means31 = fingerprint_31_gs_r2.cv_results_["mean_test_score"]
stds31 = fingerprint_31_gs_r2.cv_results_["std_test_score"]
results31 = sorted(list(zip(means31, stds31, fingerprint_31_gs_r2.cv_results_["params"])), 
                 key = lambda x: x[0],reverse=True)

In [110]:
for mean, std, params in results31[:10]:
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()

0.015 (+/-0.006) for {'model': ElasticNet(alpha=100.0, l1_ratio=0.1, random_state=0), 'model__alpha': 100.0, 'model__l1_ratio': 0.1, 'model__selection': 'cyclic'}

0.015 (+/-0.006) for {'model': ElasticNet(alpha=100.0, l1_ratio=0.1, random_state=0), 'model__alpha': 100.0, 'model__l1_ratio': 0.1, 'model__selection': 'random'}

-0.006 (+/-0.010) for {'model': ElasticNet(alpha=100.0, l1_ratio=0.1, random_state=0), 'model__alpha': 100.0, 'model__l1_ratio': 1.0, 'model__selection': 'cyclic'}

-0.006 (+/-0.010) for {'model': ElasticNet(alpha=100.0, l1_ratio=0.1, random_state=0), 'model__alpha': 100.0, 'model__l1_ratio': 1.0, 'model__selection': 'random'}

-0.007 (+/-0.010) for {'model': GradientBoostingRegressor(learning_rate=0.01, max_depth=2, n_iter_no_change=50,
                          random_state=0), 'model__learning_rate': 0.001, 'model__max_depth': 6, 'model__n_estimators': 10, 'model__n_iter_no_change': 50}

-0.007 (+/-0.008) for {'model': GradientBoostingRegressor(learning_rate=0.

In [111]:
labels31 = fingerprint_31_gs_r2.predict(X_test31)

In [112]:
r2_score(y_test31, labels31)

0.010381474692126469

### IPRI 32

In [113]:
fingerprint_32_gs_r2 = load('../files/fingerprint_32_gs_r2.joblib')
fingerprint_32_gs_mse = load('../files/fingerprint_32_gs.joblib')

In [114]:
print(fingerprint_32_gs_r2.best_params_)
print(fingerprint_32_gs_mse.best_params_)

{'model': GradientBoostingRegressor(learning_rate=0.001, max_depth=6, n_estimators=1000,
                          n_iter_no_change=50, random_state=0), 'model__learning_rate': 0.001, 'model__max_depth': 6, 'model__n_estimators': 1000, 'model__n_iter_no_change': 50}
{'model': GradientBoostingRegressor(learning_rate=0.001, max_depth=6, n_estimators=1000,
                          n_iter_no_change=50, random_state=0), 'model__learning_rate': 0.001, 'model__max_depth': 6, 'model__n_estimators': 1000, 'model__n_iter_no_change': 50}


In [115]:
means32 = fingerprint_32_gs_r2.cv_results_["mean_test_score"]
stds32 = fingerprint_32_gs_r2.cv_results_["std_test_score"]
results32 = sorted(list(zip(means32, stds32, fingerprint_32_gs_r2.cv_results_["params"])), 
                 key = lambda x: x[0],reverse=True)

In [116]:
for mean, std, params in results32[:10]:
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()

-0.011 (+/-0.047) for {'model': GradientBoostingRegressor(learning_rate=0.001, max_depth=6, n_estimators=1000,
                          n_iter_no_change=50, random_state=0), 'model__learning_rate': 0.001, 'model__max_depth': 6, 'model__n_estimators': 1000, 'model__n_iter_no_change': 50}

-0.013 (+/-0.043) for {'model': GradientBoostingRegressor(learning_rate=0.001, max_depth=6, n_estimators=1000,
                          n_iter_no_change=50, random_state=0), 'model__learning_rate': 0.001, 'model__max_depth': 6, 'model__n_estimators': 100, 'model__n_iter_no_change': 50}

-0.018 (+/-0.048) for {'model': GradientBoostingRegressor(learning_rate=0.001, max_depth=6, n_estimators=1000,
                          n_iter_no_change=50, random_state=0), 'model__learning_rate': 0.01, 'model__max_depth': 6, 'model__n_estimators': 10, 'model__n_iter_no_change': 50}

-0.018 (+/-0.025) for {'model': ElasticNet(alpha=100.0, l1_ratio=0.1, random_state=0), 'model__alpha': 100.0, 'model__l1_ratio': 1.0, 

In [111]:
labels32 = fingerprint_32_gs_r2.predict(X_test31)

In [112]:
r2_score(y_test31, labels31)

0.010381474692126469

In [83]:
#all_labels = model.predict(X_pca)

In [84]:
#pheno_df["pred_age"]=all_labels

In [85]:
#pheno_df.head()

In [86]:
#sns.relplot(x="age",y="pred_age", data=pheno_df)

In [87]:
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
scorer = make_scorer(mean_squared_error, greater_is_better=False)

In [91]:
all_gs.scoring

make_scorer(mean_squared_error, greater_is_better=False)

In [119]:
results[9][2]

{'model': Ridge(), 'model__alpha': 1.0, 'model__random_state': 0, 'model__solver': 'saga'}

In [34]:
np.logspace(1,3,3)

array([  10.,  100., 1000.])

In [120]:
model1 = ElasticNet(l1_ratio=0.01, random_state=0, alpha=1.0, selection='cyclic')
model2 = ElasticNet(l1_ratio=0.01, random_state=0, alpha=1.0, selection='random')
model3 = ElasticNet(l1_ratio=0.1, random_state=0, alpha=1.0, selection='cyclic')
model4 = ElasticNet(l1_ratio=0.1, random_state=0, alpha=1.0, selection='random')
model5 = ElasticNet(l1_ratio=0.001, random_state=0, alpha=1.0, selection='cyclic')
model6 = ElasticNet(l1_ratio=0.1, random_state=0, alpha=0.01, selection='cyclic')
model7 = ElasticNet(l1_ratio=0.001, random_state=0, alpha=1.0, selection='random')
model8 = Ridge(alpha=1.0, random_state=0,solver='lsqr')
model9 = Ridge(alpha=0.0001, random_state=0,solver='saga')
model10 = Ridge(alpha=1.0, random_state=0,solver='saga')

In [122]:
model1.fit(X_train,y_train)
model2.fit(X_train,y_train)
model3.fit(X_train,y_train)
model4.fit(X_train,y_train)
model5.fit(X_train,y_train)
model6.fit(X_train,y_train)
model7.fit(X_train,y_train)
model8.fit(X_train,y_train)
model9.fit(X_train,y_train)
model10.fit(X_train,y_train)

/Users/evb32/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 26.131092071533203, tolerance: 7.779956817626953
  model = cd_fast.enet_coordinate_descent(


Ridge(random_state=0, solver='saga')

In [125]:
model1.get_params()

{'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'l1_ratio': 0.01, 'max_iter': 1000, 'normalize': False, 'positive': False, 'precompute': False, 'random_state': 0, 'selection': 'cyclic', 'tol': 0.0001, 'warm_start': False}

In [126]:
for x in [model1, model2, model3, model4, model5, model6, model7, model8, model9, model10]:
    y_true, y_pred = y_test, x.predict(X_test)
    print(x.get_params())
    print(r2_score(y_true, y_pred))
    print()

{'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'l1_ratio': 0.01, 'max_iter': 1000, 'normalize': False, 'positive': False, 'precompute': False, 'random_state': 0, 'selection': 'cyclic', 'tol': 0.0001, 'warm_start': False}
0.7915343965610114

{'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'l1_ratio': 0.01, 'max_iter': 1000, 'normalize': False, 'positive': False, 'precompute': False, 'random_state': 0, 'selection': 'random', 'tol': 0.0001, 'warm_start': False}
0.7902410956058957

{'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'l1_ratio': 0.1, 'max_iter': 1000, 'normalize': False, 'positive': False, 'precompute': False, 'random_state': 0, 'selection': 'cyclic', 'tol': 0.0001, 'warm_start': False}
0.7382594450329123

{'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'l1_ratio': 0.1, 'max_iter': 1000, 'normalize': False, 'positive': False, 'precompute': False, 'random_state': 0, 'selection': 'random', 'tol': 0.0001, 'warm_start': False}
0.7369658805949687

{'alpha': 

In [59]:
y_true, y_pred = y_test, all_gs.predict(X_test)

In [71]:
r2_score(y_true, y_pred)

0.7915343965610114